In [1]:
import pandas as pd
import numpy as np
import os 

from datetime import datetime, timedelta
from statistics import mode


In [2]:
os.getcwd()

'C:\\Users\\Christila\\Documents\\AnimalBite'

In [3]:
loadData = pd.read_csv('Health_AnimalBites.csv')
loadData.shape


(9003, 15)

In [4]:
loadData.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc
0,1985-05-05 00:00:00,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
1,1986-02-12 00:00:00,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
2,1987-05-07 00:00:00,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
3,1988-10-02 00:00:00,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
4,1989-08-29 00:00:00,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaN,UNKNOWN,NaN,NaN,UNKNOWN


In [5]:
loadData.isnull().sum()

bite_date             317
SpeciesIDDesc         118
BreedIDDesc          5244
GenderIDDesc         2526
color                2577
vaccination_yrs      5265
vaccination_date     4888
victim_zip           1838
AdvIssuedYNDesc      6438
WhereBittenIDDesc     616
quarantine_date      6983
DispositionIDDesc    7468
head_sent_date       8608
release_date         7558
ResultsIDDesc        7460
dtype: int64

In [6]:
preprocessed_data =  loadData.copy()

In [7]:
# instead of defining exact format for datetime data mixed format will convert string to time data adaptavity. 
pd.to_datetime(preprocessed_data['bite_date'], format='mixed')

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 5013-07-15 00:00:00, at position 1384

In [8]:
#convert bite_date into needed format
preprocessed_data['bite_date'] = preprocessed_data['bite_date'].str.replace('5013-07-15 00:00:00','2013-07-15 00:00:00')

In [9]:
preprocessed_data['bite_date'] = pd.to_datetime(preprocessed_data['bite_date'])

In [10]:
# convert date into one format
pd.to_datetime(preprocessed_data['quarantine_date'], format='%Y-%m-%d %H:%M:%S')

0      1985-05-05
1      1986-02-12
2      1990-05-07
3      1990-10-02
4             NaT
          ...    
8998          NaT
8999          NaT
9000          NaT
9001          NaT
9002          NaT
Name: quarantine_date, Length: 9003, dtype: datetime64[ns]

In [18]:
# convert quarantine_date column from string type to datetime type
preprocessed_data['quarantine_date'] = pd.to_datetime(preprocessed_data['quarantine_date'])

In [12]:
preprocessed_data['quarantine_date'].dtypes

dtype('<M8[ns]')

In [13]:
#get difference between quarantine_date column and bite_date column
difference = preprocessed_data['quarantine_date'] - preprocessed_data['bite_date']

In [14]:
difference.head()

0      0 days
1      0 days
2   1096 days
3    730 days
4         NaT
dtype: timedelta64[ns]

In [15]:
# store the difference by creating new column in dataframe
preprocessed_data['difference'] = difference

In [16]:
preprocessed_data.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaN,NaN,UNKNOWN,0 days
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaN,NaN,UNKNOWN,0 days
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaN,NaN,UNKNOWN,1096 days
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaN,NaN,UNKNOWN,730 days
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaT,UNKNOWN,NaN,NaN,UNKNOWN,NaT


In [17]:
preprocessed_data[['bite_date','quarantine_date','difference']]

,bite_date,quarantine_date,difference
0,1985-05-05,1985-05-05,0 days
1,1986-02-12,1986-02-12,0 days
2,1987-05-07,1990-05-07,1096 days
3,1988-10-02,1990-10-02,730 days
4,1989-08-29,NaT,NaT
...,...,...,...
8998,2017-09-05,NaT,NaT
8999,2017-09-07,NaT,NaT
9000,2017-09-07,NaT,NaT
9001,2017-09-07,NaT,NaT


In [18]:
# select rows that are difference of the days greater than 0 days(not nulls)
selected_rows = preprocessed_data[preprocessed_data['difference']>timedelta(days=1)]

In [19]:
# sorting rows based on difference column
sorted_df = selected_rows.sort_values(by='difference',ascending=False)

In [20]:
# select needed column
sorted_df[['bite_date','quarantine_date','difference']]


,bite_date,quarantine_date,difference
2,1987-05-07,1990-05-07,1096 days
3,1988-10-02,1990-10-02,730 days
103,2010-01-27,2011-01-28,366 days
2093,2011-04-14,2011-07-15,92 days
1984,2011-04-15,2011-06-16,62 days
...,...,...,...
1075,2010-11-20,2010-11-22,2 days
1057,2010-11-13,2010-11-15,2 days
1035,2010-11-07,2010-11-09,2 days
1032,2010-11-07,2010-11-09,2 days


In [22]:
# when we need to fill null rows in quarantine_date or bite_date, first we need to find the average date.
averageDifference = preprocessed_data['difference'].mean()
averageDifference

Timedelta('-13 days +00:40:57.485029941')

In [23]:
# averageDifference cannot be a minus value. the reason can be quarantine_date lower than bite_date.
sortByDifference = preprocessed_data.sort_values(by='difference',ascending=False)
sortByDifference

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaN,NaN,UNKNOWN,1096 days
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaN,NaN,UNKNOWN,730 days
103,2010-01-27,DOG,NaN,FEMALE,BRN-BLACK,1.0,2010-07-30 00:00:00,40229,NO,BODY,2011-01-28,UNKNOWN,NaN,NaN,UNKNOWN,366 days
2093,2011-04-14,DOG,LABRADOR RETRIV,MALE,BLONDE,3.0,2009-06-02 00:00:00,40228,NO,BODY,2011-07-15,NaN,NaN,NaN,NaN,92 days
1984,2011-04-15,DOG,MALTASE,MALE,WHITE,3.0,2011-06-29 00:00:00,40245,NO,BODY,2011-06-16,NaN,NaN,NaN,NaN,62 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8998,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT
8999,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,NaT,NaN,NaN,NaN,NaN,NaT
9000,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT
9001,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,NaT,NaN,NaN,NaN,NaN,NaT


In [24]:
# we got the error data in row numbers - 2113,526,785,771,1512. we have to correct this
# in row number 1512, it cannot be the bite_date as year of 2101. it should be year of 2011
preprocessed_data['bite_date'] = preprocessed_data['bite_date'].replace(pd.to_datetime('2101-02-18'), pd.to_datetime('2011-02-18'))

# in row number 771 the month of bite_date can be 05 instead of 08
preprocessed_data['bite_date'] = preprocessed_data['bite_date'].replace(pd.to_datetime('2010-08-12'), pd.to_datetime('2010-05-12'))

In [25]:
# In row numbers  785,526 and 2113  have wrong bite_date, the difference cannot be minus value. 3 data cannot be much effect. therefore we can drop those 3 values from the dataframe
indexesToDrop = [526,785,2113]
preprocessed_data = preprocessed_data.drop(indexesToDrop)

In [26]:
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaN,NaN,UNKNOWN,0 days
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaN,NaN,UNKNOWN,0 days
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaN,NaN,UNKNOWN,1096 days
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaN,NaN,UNKNOWN,730 days
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaT,UNKNOWN,NaN,NaN,UNKNOWN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8998,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT
8999,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,NaT,NaN,NaN,NaN,NaN,NaT
9000,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT
9001,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,NaT,NaN,NaN,NaN,NaN,NaT


In [27]:
# after drop row values we need to reindex those
preprocessed_data = preprocessed_data.reset_index(drop=True)


In [28]:
#before check the average of difference we need to find the difference again
new_difference = preprocessed_data['quarantine_date'] - preprocessed_data['bite_date']
new_difference

0         0 days
1         0 days
2      1096 days
3       730 days
4            NaT
          ...   
8995         NaT
8996         NaT
8997         NaT
8998         NaT
8999         NaT
Length: 9000, dtype: timedelta64[ns]

In [29]:
# let's create a new_difference column to store the new difference
preprocessed_data['new_difference'] = new_difference

In [30]:
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaN,NaN,UNKNOWN,0 days,0 days
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaN,NaN,UNKNOWN,0 days,0 days
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaN,NaN,UNKNOWN,1096 days,1096 days
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaN,NaN,UNKNOWN,730 days,730 days
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaT,UNKNOWN,NaN,NaN,UNKNOWN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,NaT,NaN,NaN,NaN,NaN,NaT,NaT


In [31]:
sortByDifference = preprocessed_data.sort_values(by='new_difference',ascending=False)
sortByDifference

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaN,NaN,UNKNOWN,1096 days,1096 days
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaN,NaN,UNKNOWN,730 days,730 days
103,2010-01-27,DOG,NaN,FEMALE,BRN-BLACK,1.0,2010-07-30 00:00:00,40229,NO,BODY,2011-01-28,UNKNOWN,NaN,NaN,UNKNOWN,366 days,366 days
767,2010-05-12,DOG,NaN,MALE,BROWN,1.0,2010-01-04 00:00:00,40118,NO,BODY,2010-08-13,RELEASED,NaN,NaN,UNKNOWN,1 days,93 days
768,2010-05-12,DOG,NaN,FEMALE,TRI,1.0,2010-09-01 00:00:00,40229,NO,BODY,2010-08-13,RELEASED,NaN,NaN,UNKNOWN,1 days,93 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,NaT,NaN,NaN,NaN,NaN,NaT,NaT


In [32]:
# Let's again check the average of difference
average = preprocessed_data['new_difference'].mean()
average

Timedelta('3 days 16:14:23.568215892')

In [34]:
selected_cols = ['bite_date','quarantine_date','new_difference']

In [35]:
#to get any of column('bite_date','quarantine_date','difference') that have null values
null_rows = preprocessed_data[preprocessed_data[selected_cols].isnull().sum(axis=1) >= 1]
null_rows

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaT,UNKNOWN,NaN,NaN,UNKNOWN,NaT,NaT
5,1989-11-24,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40211,NO,BODY,NaT,UNKNOWN,NaN,NaN,UNKNOWN,NaT,NaT
71,2010-01-09,DOG,NaN,UNKNOWN,BLK-WHT,NaN,NaN,40219,NO,BODY,NaT,UNKNOWN,2010-01-10 00:00:00,NaN,UNKNOWN,NaT,NaT
78,2010-01-12,DOG,NaN,MALE,BRN-WHT,NaN,NaN,40214,NO,BODY,NaT,UNKNOWN,2010-01-14 00:00:00,NaN,UNKNOWN,NaT,NaT
121,2010-02-04,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40229,NO,BODY,NaT,UNKNOWN,2010-02-05 00:00:00,NaN,UNKNOWN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,NaT,NaN,NaN,NaN,NaN,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,NaT,NaN,NaN,NaN,NaN,NaT,NaT


In [36]:
#select only needed columns
null_rows[selected_cols]

,bite_date,quarantine_date,new_difference
4,1989-08-29,NaT,NaT
5,1989-11-24,NaT,NaT
71,2010-01-09,NaT,NaT
78,2010-01-12,NaT,NaT
121,2010-02-04,NaT,NaT
...,...,...,...
8995,2017-09-05,NaT,NaT
8996,2017-09-07,NaT,NaT
8997,2017-09-07,NaT,NaT
8998,2017-09-07,NaT,NaT


In [37]:
# create a copy of null_rows
copyOfNullRows = null_rows.copy()

In [38]:
columns_of_interest = ['bite_date', 'quarantine_date']


In [39]:
null_rows_indexes = preprocessed_data[preprocessed_data[columns_of_interest].isnull().any(axis=1)].index


In [40]:
null_rows_indexes

Index([   4,    5,   71,   78,  121,  145,  169,  170,  180,  181,
       ...
       8990, 8991, 8992, 8993, 8994, 8995, 8996, 8997, 8998, 8999],
      dtype='int64', length=6999)

In [61]:
# edit quarantine_date when have only bite_date. to edit this we need to add average date difference from quarantine_date.
preprocessed_data['quarantine_date'] = preprocessed_data['quarantine_date'].fillna(preprocessed_data['bite_date'] + pd.Timedelta(days=3))

In [62]:
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaT,NaN,UNKNOWN,1096 days,1096 days,NaT
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaT,NaN,UNKNOWN,730 days,730 days,NaT
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,1989-09-01,UNKNOWN,NaT,NaN,UNKNOWN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [63]:
# edit bite_date when have only quarantine_date. to edit this we need to substract average date difference from quarantine_date.
preprocessed_data['bite_date'] = preprocessed_data['bite_date'].fillna(preprocessed_data['quarantine_date'] - pd.Timedelta(days=3))

In [64]:
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaT,NaN,UNKNOWN,1096 days,1096 days,NaT
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaT,NaN,UNKNOWN,730 days,730 days,NaT
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,1989-09-01,UNKNOWN,NaT,NaN,UNKNOWN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [65]:
# now we can check the rest of rows that have null data in both column
restOfNullRows = preprocessed_data[preprocessed_data[selected_cols].isnull().sum(axis=1) > 1]
restOfNullRows

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
1511,NaT,OTHER,NaN,NaN,NaN,NaN,NaN,40299,NO,NaN,NaT,DIED,2011-05-24,NaN,NEGATIVE,NaT,NaT,NaT
1512,NaT,BAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2011-08-09,NaN,NEGATIVE,NaT,NaT,NaT
1513,NaT,BAT,NaN,NaN,NaN,NaN,NaN,40215,NaN,NaN,NaT,NaN,2015-02-28,NaN,NEGATIVE,NaT,NaT,NaT
1514,NaT,BAT,NaN,NaN,NaN,NaN,NaN,40214,NaN,NaN,NaT,NaN,2015-03-01,NaN,NEGATIVE,NaT,NaT,NaT
1515,NaT,CAT,NaN,NaN,NaN,NaN,NaN,40214,NaN,BODY,NaT,NaN,2015-03-05,NaN,NEGATIVE,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,NaT,DOG,BORDER COLLIE,MALE,BLK WHT,1.0,2017-07-18 00:00:00,NaN,NaN,NaN,NaT,NaN,NaT,2017-08-18 00:00:00,NaN,NaT,NaT,NaT
8921,NaT,DOG,NaN,NaN,NaN,NaN,NaN,40206,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8924,NaT,BAT,NaN,NaN,NaN,NaN,NaN,40291,YES,NaN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8925,NaT,BAT,NaN,NaN,NaN,NaN,NaN,40291,YES,NaN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [56]:
# before filling rest of null date in bite_date and quarantine_date column, let's check the diffence and average date between head_sent_date and bite_date
differnceOfBiteAndHeadSent = preprocessed_data['head_sent_date']- preprocessed_data['bite_date']
preprocessed_data['differnceOfBiteAndHeadSent'] = differnceOfBiteAndHeadSent

TypeError: cannot subtract DatetimeArray from ndarray

In [57]:
# before substract we need to convert head_sent_date column into datetime
preprocessed_data['head_sent_date'] = pd.to_datetime(preprocessed_data['head_sent_date'])


In [59]:
differnceOfBiteAndHeadSent = preprocessed_data['head_sent_date']- preprocessed_data['bite_date']
preprocessed_data['differnceOfBiteAndHeadSent'] = differnceOfBiteAndHeadSent

In [60]:
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaT,NaN,UNKNOWN,1096 days,1096 days,NaT
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaT,NaN,UNKNOWN,730 days,730 days,NaT
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaT,UNKNOWN,NaT,NaN,UNKNOWN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [66]:
avgDiffernceOfBiteAndHeadSent = preprocessed_data['differnceOfBiteAndHeadSent'].mean()

In [67]:
avgDiffernceOfBiteAndHeadSent

Timedelta('8 days 18:21:10.588235294')

In [70]:
# now let's check that have a value in head_sent_date column.
columnsInCount = ['bite_date','head_sent_date']
nullRowsinHeadSentDate = preprocessed_data[preprocessed_data[columnsInCount].isnull().sum(axis=1) >= 1]


In [73]:
nullRowsinHeadSentDate

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaT,NaN,UNKNOWN,1096 days,1096 days,NaT
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaT,NaN,UNKNOWN,730 days,730 days,NaT
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,1989-09-01,UNKNOWN,NaT,NaN,UNKNOWN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [72]:
preprocessed_data.sort_values(by='differnceOfBiteAndHeadSent',ascending=False)


,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
1405,2014-03-24,DOG,PIT BULL,NaN,WHITE,NaN,NaN,40205,NaN,BODY,2014-03-27,NaN,2017-03-24,NaN,NEGATIVE,NaT,NaT,1096 days
1432,2015-06-23,DOG,PIT BULL,FEMALE,BRINDLE,NaN,NaN,40215,NaN,BODY,2015-06-26,NaN,2016-06-26,NaN,NEGATIVE,NaT,NaT,369 days
765,2010-05-12,CAT,NaN,UNKNOWN,ORANGE,NaN,NaN,40216,NO,BODY,2010-05-15,UNKNOWN,2010-08-13,NaN,UNKNOWN,NaT,NaT,93 days
764,2010-05-12,CAT,NaN,UNKNOWN,ORANGE-WHT,NaN,NaN,40203,NO,HEAD,2010-05-15,UNKNOWN,2010-08-12,NaN,UNKNOWN,NaT,NaT,92 days
1434,2015-07-14,DOG,PIT BULL,FEMALE,BLUE WHIT,NaN,NaN,40219,NaN,BODY,2015-07-17,NaN,2015-07-24,NaN,NaN,NaT,NaT,10 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [75]:
# when looking at sort values in differnceOfBiteAndHeadSent column, values cannot be 1096, 369, 93,92, days. those values should be change
# when looking at the real dataset, row of 1404 row is filled in 2014. There for 1405 row also should be in 2014. therefore we need correct head_sent_date in R1404 into 2014. 

preprocessed_data.loc[1405, 'head_sent_date'] = pd.to_datetime('2014-03-24')

In [77]:
# when looking at the real dataset, row of 1431 and 1433 row is filled in 2015. There for 1432 row also should be in 2015. therefore we need to correct head_sent_date in R1432 into 2015. 

preprocessed_data.loc[1432, 'head_sent_date'] = pd.to_datetime('2015-06-26')

In [79]:
# we need to change these location also 764,765
preprocessed_data.loc[764, 'bite_date'] = pd.to_datetime('2010-08-12')
preprocessed_data.loc[765, 'bite_date'] = pd.to_datetime('2010-08-12')

In [80]:
preprocessed_data.sort_values(by='differnceOfBiteAndHeadSent',ascending=False)

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
1405,2014-03-24,DOG,PIT BULL,NaN,WHITE,NaN,NaN,40205,NaN,BODY,2014-03-27,NaN,2014-03-24,NaN,NEGATIVE,NaT,NaT,1096 days
1432,2015-06-23,DOG,PIT BULL,FEMALE,BRINDLE,NaN,NaN,40215,NaN,BODY,2015-06-26,NaN,2015-06-26,NaN,NEGATIVE,NaT,NaT,369 days
765,2010-08-12,CAT,NaN,UNKNOWN,ORANGE,NaN,NaN,40216,NO,BODY,2010-05-15,UNKNOWN,2010-08-13,NaN,UNKNOWN,NaT,NaT,93 days
764,2010-08-12,CAT,NaN,UNKNOWN,ORANGE-WHT,NaN,NaN,40203,NO,HEAD,2010-05-15,UNKNOWN,2010-08-12,NaN,UNKNOWN,NaT,NaT,92 days
1434,2015-07-14,DOG,PIT BULL,FEMALE,BLUE WHIT,NaN,NaN,40219,NaN,BODY,2015-07-17,NaN,2015-07-24,NaN,NaN,NaT,NaT,10 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [82]:
preprocessed_data['differnceOfBiteAndHeadSent'] = preprocessed_data['head_sent_date']- preprocessed_data['bite_date']
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaT,NaN,UNKNOWN,1096 days,1096 days,NaT
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaT,NaN,UNKNOWN,730 days,730 days,NaT
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,1989-09-01,UNKNOWN,NaT,NaN,UNKNOWN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [83]:
#let's again check the mean value
avgDiffernceOfBiteAndHeadSent = preprocessed_data['differnceOfBiteAndHeadSent'].mean()
avgDiffernceOfBiteAndHeadSent

Timedelta('1 days 07:36:06.515837104')

In [84]:
# there are some values in the row of head_sent_date column. by using those values we can fill bite_date column that are having null values in the rows
preprocessed_data['bite_date'] = preprocessed_data['bite_date'].fillna( preprocessed_data['head_sent_date'] - pd.Timedelta(days=1))


In [86]:
preprocessed_data[preprocessed_data[selected_cols].isnull().sum(axis=1) > 1]

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
1511,2011-05-23,OTHER,NaN,NaN,NaN,NaN,NaN,40299,NO,NaN,NaT,DIED,2011-05-24,NaN,NEGATIVE,NaT,NaT,NaT
1512,2011-08-08,BAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,2011-08-09,NaN,NEGATIVE,NaT,NaT,NaT
1513,2015-02-27,BAT,NaN,NaN,NaN,NaN,NaN,40215,NaN,NaN,NaT,NaN,2015-02-28,NaN,NEGATIVE,NaT,NaT,NaT
1514,2015-02-28,BAT,NaN,NaN,NaN,NaN,NaN,40214,NaN,NaN,NaT,NaN,2015-03-01,NaN,NEGATIVE,NaT,NaT,NaT
1515,2015-03-04,CAT,NaN,NaN,NaN,NaN,NaN,40214,NaN,BODY,NaT,NaN,2015-03-05,NaN,NEGATIVE,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,NaT,DOG,BORDER COLLIE,MALE,BLK WHT,1.0,2017-07-18 00:00:00,NaN,NaN,NaN,NaT,NaN,NaT,2017-08-18 00:00:00,NaN,NaT,NaT,NaT
8921,NaT,DOG,NaN,NaN,NaN,NaN,NaN,40206,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8924,NaT,BAT,NaN,NaN,NaN,NaN,NaN,40291,YES,NaN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8925,NaT,BAT,NaN,NaN,NaN,NaN,NaN,40291,YES,NaN,NaT,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [88]:
# when see the dataset, lot of data already filled.. the rest of the data we can fill by using previous dates.
preprocessed_data['bite_date'] = preprocessed_data['bite_date'].fillna(method='ffill')


C:\Users\Christila\AppData\Local\Temp\ipykernel_14752\1180076872.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  preprocessed_data['bite_date'] = preprocessed_data['bite_date'].fillna(method='ffill')


In [90]:
# now we can again edit the 'quatantine_date' column by using bite_date and average of difference those two columns
preprocessed_data['quarantine_date'] = preprocessed_data['quarantine_date'].fillna(preprocessed_data['bite_date'] + pd.Timedelta(days=3))

In [92]:
preprocessed_data['quarantine_date'].isnull().sum()

0

In [93]:
preprocessed_data

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc,difference,new_difference,differnceOfBiteAndHeadSent
0,1985-05-05,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
1,1986-02-12,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12,UNKNOWN,NaT,NaN,UNKNOWN,0 days,0 days,NaT
2,1987-05-07,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07,UNKNOWN,NaT,NaN,UNKNOWN,1096 days,1096 days,NaT
3,1988-10-02,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02,UNKNOWN,NaT,NaN,UNKNOWN,730 days,730 days,NaT
4,1989-08-29,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,1989-09-01,UNKNOWN,NaT,NaN,UNKNOWN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,2017-09-05,DOG,NaN,NaN,NaN,NaN,NaN,40243,NaN,UNKNOWN,2017-09-08,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8996,2017-09-07,DOG,POMERANIAN,MALE,RED,NaN,NaN,40204,NaN,HEAD,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8997,2017-09-07,DOG,LABRADOR RETRIV,MALE,BROWN,NaN,NaN,47130,NaN,UNKNOWN,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT
8998,2017-09-07,DOG,LABRADOR RETRIV,FEMALE,BLK WHT,NaN,NaN,40229,NaN,BODY,2017-09-10,NaN,NaT,NaN,NaN,NaT,NaT,NaT


In [94]:
preprocessed_data['SpeciesIDDesc'].isnull().sum()

118

In [115]:
# get the most bitten animalin the dataset

mode(preprocessed_data['SpeciesIDDesc'])

'DOG'

In [98]:
#lets fill missing values in 'SpeciesIDDesc' column using mode value.
preprocessed_data['SpeciesIDDesc'] = preprocessed_data['SpeciesIDDesc'].fillna('DOG')


In [99]:
preprocessed_data['SpeciesIDDesc'].isnull().sum()

0

In [100]:
# let's check gender column
preprocessed_data['GenderIDDesc'].isnull().sum()


2525

In [116]:
# categorical_values = preprocessed_data['SpeciesIDDesc']
mode(preprocessed_data['GenderIDDesc'])


'MALE'

In [106]:
# let's fill null values using mode of gender
preprocessed_data['GenderIDDesc'] = preprocessed_data['GenderIDDesc'].fillna('MALE')


In [109]:
preprocessed_data['GenderIDDesc'] = preprocessed_data['GenderIDDesc'].replace('UNKNOWN','MALE')

In [110]:
preprocessed_data['GenderIDDesc']

0       FEMALE
1         MALE
2         MALE
3         MALE
4       FEMALE
         ...  
8995      MALE
8996      MALE
8997      MALE
8998    FEMALE
8999      MALE
Name: GenderIDDesc, Length: 9000, dtype: object

In [111]:
preprocessed_data['GenderIDDesc'].value_counts()

GenderIDDesc
MALE      6984
FEMALE    2016
Name: count, dtype: int64

In [113]:
#let's get unique count in color column
preprocessed_data['color'].value_counts()

color
BLACK         671
BROWN         433
WHITE         394
BLK WHT       277
TAN           223
             ... 
WH/BLACK        1
BL/WH/BROW      1
ORG/WHITE       1
TAN/SILVER      1
BRONW           1
Name: count, Length: 712, dtype: int64

In [19]:
mode(preprocessed_data['color'].dropna())

'BLACK'

In [117]:
preprocessed_data['color'].isnull().sum()

2576

In [20]:
# we can use these null values using second mode value.. we can see that from below table
preprocessed_data['color'] = preprocessed_data['color'].fillna('BLACK')


In [21]:
mode(preprocessed_data['color'])

'BLACK'

In [22]:
preprocessed_data['color']

0       LIG. BROWN
1        BRO & BLA
2            BLACK
3        BLA & BRO
4          BLK-WHT
           ...    
8998         BLACK
8999           RED
9000         BROWN
9001       BLK WHT
9002       BRN BLK
Name: color, Length: 9003, dtype: object

In [ ]:
preprocessed_data['color'].isna()